# PR-028 DenseNet

김성훈 님의 [DenseNet 강의](https://www.youtube.com/watch?v=fe2Vn0mwALI) 감사드립니다.

CIFAR10 데이터를 DenseNet 으로 학습시킵니다.

논문: https://arxiv.org/abs/1608.06993

### Data Set
CIFAR-10 데이터는 기본적으로 케라스에서 제공해 줍니다.

In [1]:
from keras.datasets import cifar10

import numpy as np
from keras.utils.np_utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# normalization
x_train = np.array(x_train) / 127.5 - 1
x_test = np.array(y_test) / 127.5 - 1

# one-hot encoding
y_train = np.array(y_train)
y_test = np.array(y_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Using TensorFlow backend.


### Build Model

논문에서는 크게 아래 사진과 같이 4개의 구조를 만들었습니다.

![image.png](DenseNet.png)

DenseNet-121 을 조금만 변형해서 사용해 봅니다.

input_shape 를 (32, 32, 3) 으로 입력시켜 주고

끝에 붙는 fully connected layer 의 크기를 10 으로 사용합니다.

In [2]:
from keras.models import Model
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras.layers import Conv2D, Activation, Input, Dense
from keras.layers.pooling import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D

# hyperparameters
K = 32
init_ch = 64         # 가장 첫 layer의 채널 수 입니다.
compression = 0.5    # transition layer 에서의 channel을 얼마나 줄일 것인지 비율입니다.
inp_shape = (32, 32, 3)


# conv_layer를 간편하게 쓰기 위해 wrapper 입니다.
def conv_layer(x, filters, k, s):
    return Conv2D(filters,
                  kernel_size=(k, k),
                  strides=(s, s),
                  padding='same',
                  kernel_regularizer=l2(0.01))(x)

def dense_block(x, dense_block_size):
    for i in range(dense_block_size):
        tmp = x
        
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = conv_layer(x, 4*K, 1, 1)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = conv_layer(x, K, 3, 1)
        
        x = concatenate([x, tmp])
        
    return x

def transition_block(x):
    shape = x.get_shape().as_list()                    # output 의 shape을 가져옵니다.
    compressed_channels = int(shape[3] * compression)  # compression 비율만큼 channel을 줄여 줍니다.
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = conv_layer(x, compressed_channels, 1, 1)
    x = AveragePooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    return x

input_img = Input(inp_shape)

x = conv_layer(input_img, init_ch, 7, 2)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

x = dense_block(x, 6)
x = transition_block(x)

x = dense_block(x, 12)
x = transition_block(x)

x = dense_block(x, 24)
x = transition_block(x)

x = dense_block(x, 16)

x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='softmax')(x)

dense121 = Model(input_img, x)

dense121.summary() 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 16, 16, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 16, 16, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

### Train Model

In [5]:
dense121.compile(optimizer='adam', loss='categorical_crossentropy')

history = dense121.fit(x_train, y_train, epochs=30, batch_size=32)

Epoch 1/30
50000/50000 [==============================] - 1762s 35ms/step - loss: 1.9528
Epoch 2/30
50000/50000 [==============================] - 1748s 35ms/step - loss: 1.7774
Epoch 3/30
50000/50000 [==============================] - 1748s 35ms/step - loss: 1.6949
Epoch 4/30
50000/50000 [==============================] - 1748s 35ms/step - loss: 1.5447
Epoch 5/30
50000/50000 [==============================] - 1749s 35ms/step - loss: 1.4656
Epoch 6/30
50000/50000 [==============================] - 1748s 35ms/step - loss: 1.4170
Epoch 7/30
50000/50000 [==============================] - 1749s 35ms/step - loss: 1.3704
Epoch 8/30
50000/50000 [==============================] - 1748s 35ms/step - loss: 1.3459
Epoch 9/30
50000/50000 [==============================] - 1748s 35ms/step - loss: 1.3130
Epoch 10/30
50000/50000 [==============================] - 1750s 35ms/step - loss: 1.2955
Epoch 11/30
50000/50000 [==============================] - 1749s 35ms/step - loss: 1.2712
Epoch 12/30
50000/5

KeyboardInterrupt: 

## Contact me
케라스를 사랑하는 개발자 입니다.

질문, 조언, contribtuion 등 소통은 언제나 환영합니다.

Anthony Kim(김동현) : artit.anthony@gmail.com